In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./resources/processed/v3/train_noise_remove_llm3.csv', index_col=0)

In [3]:
df

,ID,text,target,target_name,re_text
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,IT과학,"복원: 정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)\n\n주..."
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,사회,"복원: 경찰, '국회 불법 로비' 한어총 회장 등 20명 송치\n\n주제: 정치\n..."
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,정치,"복원: 김정은 ""자주통일 새시대 열어나가야""\n\n주제: 정치\n\n설명: 이 문장..."
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,경제,복원: 갤노트8 주말 27만대 개통...시장은 불법 보조금 얼룩\n\n주제: 경제\...
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,세계,"복원: 軍 ""美대선 앞두고 北 무수단 발사 대비해 감시 강화""\n\n주제: 세계\n..."
...,...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,세계,복원: 트럼프 폭스뉴스 앵커들 충성도 점수 매겨...10점 만점에 12점도\n\n주...
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,정치,복원: 삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠\n\n주제: IT과학\n\...
2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4,IT과학,복원: 텔레그램 등 아시아서 2시간 다운...카카오뱅크 서비스 장애 39분 지속(2...
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,스포츠,"복원: 인터뷰: 류현진, 친구에게 안타 맞는 것 싫어해...승부는 냉정\n\n주제:..."


In [27]:
def calculate_text_diff(text, re_text):
    t1 = text.replace('...','…')
    t1 = t1.replace(' ','')
        
    re_text = re_text.replace('...','…')
    re_text = re_text.replace(' ','')
    
    return len(set(t1) & set(re_text)) / len(set(t1))

noist_lst = []
lst = []
for _, row in df.iterrows():
    id = row['ID']
    target = row['target']
    target_name = row['target_name']
    text = row['text']
    re_text = row['re_text'].split('\n')[0].split('복원:')[-1].strip()
    topic_gen = row['re_text'].split('주제:')[1].split('\n')[0]
    
    score = calculate_text_diff(text, re_text)
    # if score < 0.695:
    if score < 0.83:
        noist_lst.append([id, text, target, target_name, re_text, topic_gen])
    else:
        lst.append([id, text, target, target_name, re_text, topic_gen])

In [28]:
df_noise = pd.DataFrame(noist_lst, columns = ['ID', 'text', 'target', 'target_name', 're_text', 'gen_target'])

In [29]:
df_noise['gen_target'] = [x.strip() for x in df_noise['gen_target']]

In [34]:
df_noise

,ID,text,target,target_name,re_text,gen_target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,IT과학,"정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)",IT과학
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,사회,"경찰, '국회 불법 로비' 한어총 회장 등 20명 송치",정치
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,정치,"김정은 ""자주통일 새시대 열어나가야""",정치
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,세계,"軍 ""美대선 앞두고 北 무수단 발사 대비해 감시 강화""",세계
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,스포츠,프로야구 롯데-KIA 광주 경기 우천취소,스포츠
...,...,...,...,...,...,...
1587,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0,생활문화,남원소식 춘향단장 모집,생활문화
1588,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2,정치,"이총리, 세출 예산 완전 집행 못해 할 일 꽤 남아",경제
1589,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1,스포츠,높은 취업률...선수 몸값 상승 논란,경제
1590,ynat-v1_train_02793,경찰 5월초 유커와 日관광객위해 바가지 요금 집중 단속,1,스포츠,경찰 5월 초 외국인 관광객을 위해 바가지 요금 집중 단속,생활문화


In [35]:
sum(df_noise['target_name'] == df_noise['gen_target']) / len(df_noise)

0.5483668341708543

In [36]:
# df_noise.to_csv('train_noise.csv', encoding='utf-8-sig', index=0)
df_noise.to_csv('train_noise2.csv', encoding='utf-8-sig', index=0)

In [32]:
df_not_noise = pd.DataFrame(lst, columns = ['ID', 'text', 'target', 'target_name', 're_text', 'gen_target'])

In [37]:
df_not_noise

,ID,text,target,target_name,re_text,gen_target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,경제,갤노트8 주말 27만대 개통...시장은 불법 보조금 얼룩,경제
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,생활문화,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,경제
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,4,IT과학,아가메즈 33득점 우리카드 KB손해보험 완파...3위 굳혀,스포츠
3,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6,세계,박근혜 대통령 얼마나 많이 놀라셨어요...경주 지진현장 방문 종합,정치
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4,IT과학,듀얼심 아이폰 하반기 출시설 솔솔...알뜰폰 기대감,IT과학
...,...,...,...,...,...,...
1203,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2,정치,"문재인 대통령, 김기식 금감원장 사표 수리키로(종합)",정치
1204,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,세계,트럼프 폭스뉴스 앵커들 충성도 점수 매겨...10점 만점에 12점도,정치
1205,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,정치,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,IT과학
1206,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,스포츠,"인터뷰: 류현진, 친구에게 안타 맞는 것 싫어해...승부는 냉정",스포츠


In [38]:
# df_not_noise.to_csv('train_not_noise.csv', encoding='utf-8-sig', index=0)
df_noise.to_csv('train_not_noise2.csv', encoding='utf-8-sig', index=0)